In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
CheckinInfo=pd.read_csv('/content/gdrive/My Drive/Dataset/checkin_bj_uniq.txt',header=None,sep='\t')
CheckinInfo.columns=['User_id','Time','latitude','longititude','Item_id']
CheckinInfo = CheckinInfo.loc[:74960]
#1000 - 74960
#5000 - 337240
#10000 - 655759
#20000 - 1263635
#30000 - 1827668
#40000 - 2432870
#50000 - 3093551
CheckinInfo.head()

,User_id,Time,latitude,longititude,Item_id
0,1000030044,2011/10/11 17:01:59,36.677125,117.023793,j:3D6A0B8EF0FB83BD
1,1000030044,2011/10/15 14:33:07,40.167353,116.565249,j:351926FDF2BF794A
2,1000030044,2011/10/20 16:47:18,39.895401,116.322000,j:EB34D7A2DCA7F021
3,1000030044,2011/10/26 12:38:23,38.043217,114.481255,j:7C22E78A6E6BC3FFD7807B30A00D0CF2
4,1000030044,2011/10/29 10:33:20,39.953124,116.492071,j:8DFC0EDFDD5E6858


In [0]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

a1=CheckinInfo.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()
a1.User_id = a1.User_id.astype('int64')

person_c = CategoricalDtype(sorted(a1.User_id.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(a1.Item_id.unique()), ordered=True)

items=sorted(a1["Item_id"].unique())
user_labels=list(sorted(a1.User_id.unique()))

In [0]:
d_user = {}
d_item = {}
count = 0
for i in a1.User_id.unique():
    d_user[i]=count
    d_user[-count] = i
    count+= 1
count = 0
for i in a1.Item_id.unique():
    d_item[i]=count
    d_item[count] = i
    count+= 1

In [0]:
#Creation

In [0]:
#generating single user dictionary

User_place_count=a1.groupby( [ "User_id"] ).size().to_frame(name = 'count').reset_index()
User_place_count.count = User_place_count["count"].astype('int64')
User_place_count.User_id = User_place_count["User_id"].astype('int64')


dict_singleUser=[]
for index,row in User_place_count[:][User_place_count["count"] == 1].iterrows():
    dict_singleUser.append(row.User_id)

In [0]:
dict_singleUser2=[]
for i in dict_singleUser:
    if CheckinInfo[:][CheckinInfo.User_id == i].User_id.count() == 1:
      dict_singleUser2.append(i)

In [0]:
# hitrate - one visit hide
from sklearn.model_selection import train_test_split

temp = CheckinInfo[~CheckinInfo.User_id.isin(dict_singleUser2)]
train, test = train_test_split(temp, test_size=0.1)
train = CheckinInfo.drop(test.index.values.tolist())

a1_train=train.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()
a1_test=test.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()

with open('/content/gdrive/My Drive/Dataset/matlab_input/train_now.txt','w') as f:
    for index,row in a1_train.iterrows():
        f.write(str(d_user[row["User_id"]]) +'\t'+str(d_item[row["Item_id"]]) +'\t'+str(row["count"])+'\n')
        

with open('/content/gdrive/My Drive/Dataset/matlab_input/test_now.txt','w') as f:
    for index,row in a1_test.iterrows():
        f.write(str(d_user[row["User_id"]]) +'\t'+str(d_item[row["Item_id"]]) +'\t'+str(row["count"])+'\n')
        
print("Testing Sample2 % :",(test.User_id.count()/CheckinInfo.User_id.count())*100)

Testing Sample2 % : 9.98385827296861


In [0]:
# hitrate - one visit hide
row = a1_train.User_id.astype(person_c).cat.codes
col = a1_train.Item_id.astype(thing_c).cat.codes
sparse_checkin = csr_matrix((a1_train['count'], (row, col)),shape=(person_c.categories.size, thing_c.categories.size))

In [0]:
# hitrate - one item hide
import random
d1={} #all user-item
d2={} #hidden user-item

User_place_count=a1.groupby( [ "User_id"] ).size().to_frame(name = 'count').reset_index()
User_place_count.count = User_place_count["count"].astype('int64')
User_place_count.User_id = User_place_count["User_id"].astype('int64')

#generating single user dictionary
dict_singleUser={}
for index,row in User_place_count[:][User_place_count["count"] == 1].iterrows():
    dict_singleUser[row.User_id] = 0;
    
#generating d[user]=item(s)
for index,row in a1.iterrows():
    key1=row.User_id
    key2=row.Item_id
    if key1 not in dict_singleUser.keys():
        if key1 not in d1.keys():
            d1[key1]=[key2]
        else:
            d1[key1].append(key2)

In [0]:
# hitrate - one item hide
import tqdm
list_toBeDeleted=[]
visited=[]

with tqdm.tqdm(total=a1.User_id.count()) as progress:
  for index,row in a1.iterrows():
      if row.User_id not in visited and row.User_id not in dict_singleUser:
          temp=random.choice(d1[row.User_id])
          d2[row.User_id]=temp
          d1[row.User_id].remove(temp)
          visited.append(row.User_id)
          list_toBeDeleted.append(index)
      progress.update(1)

100%|███████████████████████████████████████████████████████████████████████| 1992299/1992299 [37:35<00:00, 883.13it/s]


In [0]:
# hitrate - one item hide
a1.drop(list_toBeDeleted, inplace=True)
rows = a1.User_id.astype(person_c).cat.codes
cols = a1.Item_id.astype(thing_c).cat.codes

with open('/content/gdrive/My Drive/Dataset/matlab_input/train1.txt','w') as f:
    for index,row in a1.iterrows():
        f.write(str(d_user[row["User_id"]]) +'\t'+str(d_item[row["Item_id"]]) +'\t'+str(row["count"])+'\n')
        

with open('/content/gdrive/My Drive/Dataset/matlab_input/test1.txt','w') as f:
    for key in d2.keys():
        f.write(str(d_user[key]) +'\t'+str(d_item[d2[key]]) +'\t'+"1"+'\n')
        
        
sparse_checkin = csr_matrix((a1['count'], (rows, cols)),shape=(person_c.categories.size, thing_c.categories.size))

In [0]:
#Evaluation

In [0]:
# hitrate - one item hide
d2 = {}
dict_test={}
list_correct2=[]

with open("/content/gdrive/My Drive/Dataset/matlab_input/test_now.txt") as f:
    for line in f:
        key1,key2,key3=line.split()
        key1u=d_user[-int(key1)]
        key2i=d_item[int(key2)]
        if key1u not in d2.keys():
            d2[key1u]=[key2i]
        else:
            d2[key1u].append(key2i)
            
with open("/content/gdrive/My Drive/Dataset/matlab_input/results.txt") as g:
    for line in g:
        key1,key2=line.split()
        key1u=d_user[-(int(key1)-1)]
        key2i=d_item[int(key2)-1]
        if key1u not in dict_test.keys():
            dict_test[key1u]=[key2i]
        else:
            dict_test[key1u].append(key2i)
          
hit=0
print("\n----------------------------\n")
for key,value in dict_test.items():
    if key in d2.keys():
        for item in value:
            if item in d2[key]:
                hit+=1
                list_correct2.append(key)
total=len(d2.keys())
print("\nHit:",hit,"\nHit Ratio:",hit/total)
print(list_correct2)


----------------------------


Hit: 7451 
Hit Ratio: 0.15937967914438503
[35277, 103500, 120430, 21666866, 22313121, 36319462, 62868768, 71858195, 72515213, 80715715, 220794239, 270594044, 1000569415, 1002653642, 1002850984, 1003485590, 1007977751, 1010376130, 1011221801, 1011474317, 1012425625, 1013147604, 1013246317, 1013463552, 1014296543, 1015243577, 1018421181, 1018891000, 1018946971, 1020168675, 1021054951, 1022678640, 1022985214, 1023499484, 1023638752, 1024570371, 1024723671, 1024931364, 1025746490, 1025836422, 1025964427, 1026680300, 1026897843, 1026936315, 1028612147, 1029584980, 1029667770, 1029866321, 1030854517, 1031187417, 1031944881, 1033184385, 1033341722, 1033343245, 1033537014, 1035778783, 1036354821, 1036358562, 1036595281, 1036947391, 1037316943, 1037575342, 1037607175, 1039014865, 1039038550, 1040559602, 1041721064, 1042147850, 1042398754, 1042458022, 1042655072, 1042696444, 1042854620, 1042921152, 1043351814, 1043463903, 1043858344, 1043891372, 1043903183, 104419

In [0]:
# hitrate - one visit hide
dict_test={}
list_correct2=[]
d2={} 

with open("/content/gdrive/My Drive/Dataset/matlab_input/test_now.txt",'r') as f:
    for line in f:
        key1,key2,key3=line.split()
        key1u=d_user[-int(key1)]
        key2i=d_item[int(key2)]
        if key1u not in d2.keys():
            d2[key1u]=[key2i]
        else:
            d2[key1u].append(key2i)
            
with open("/content/gdrive/My Drive/Dataset/matlab_input/results@100.txt",'r') as g:
    for line in g:
        key1,key2=line.split()
        key1u=d_user[-(int(key1)-1)]
        key2i=d_item[int(key2)-1]
        if key1u not in dict_test.keys():
            dict_test[key1u]=[key2i]
        else:
            dict_test[key1u].append(key2i)
hit=0
print("\n----------------------------\n")
for key,value in dict_test.items():
    if key in d2.keys():
        for item in value:
            if item in d2[key]:
                hit+=1
                list_correct2.append(key)

total=len(d2.keys())
print("\nHit:",hit,"\nHit Ratio:",hit/total)
print(list_correct2)


----------------------------


Hit: 351 
Hit Ratio: 0.5416666666666666
[104899, 1000569415, 1000588815, 1001449731, 1001495634, 1001740393, 1001758303, 1002323777, 1002323777, 1002647827, 1002797063, 1002797063, 1002797063, 1003223223, 1003223223, 1003572170, 1003572170, 1003572170, 1003606177, 1003606177, 1003606177, 1003606177, 1004172250, 1004333025, 1005988114, 1005988114, 1005988114, 1007220330, 1008739351, 1008792997, 1009001235, 1009001235, 1009001235, 1009406375, 1009570137, 1009570137, 1009570137, 1009819810, 1010007160, 1010007160, 1010007160, 1010036244, 1011318050, 1013463552, 1013896037, 1013896037, 1013950032, 1014033540, 1014033540, 1014251693, 1014531355, 1014730831, 1015222123, 1015262311, 1015763431, 1016762742, 1016961197, 1017132760, 1018453074, 1018453074, 1018583713, 1018583713, 1018912647, 1018921635, 1018921635, 1020168675, 1021052367, 1021054951, 1021876263, 1022248230, 1022855285, 1022855285, 1023803102, 1024380972, 1024571947, 1024723671, 1024846795, 1024846

In [0]:
20
Hit: 306 
Hit Ratio: 0.4722222222222222
  
30
Hit: 324 
Hit Ratio: 0.5
 
50
Hit: 329 
Hit Ratio: 0.5077160493827161
  
70
Hit: 332 
Hit Ratio: 0.5123456790123457
  
90
Hit: 329 
Hit Ratio: 0.5077160493827161
  
100
Hit: 351 
Hit Ratio: 0.5416666666666666
